In [2]:
from pm4py.objects.log.importer.xes import importer as xes_import_factory
import pm4py

from pandas.api.types import is_numeric_dtype
import pingouin as pg

import pandas as pd
import numpy as np

In [ ]:
under_60 = xes_import_factory.apply('../Event Logs/under_60.xes/under_60.xes')
over_60 = xes_import_factory.apply('../Event Logs/over_60.xes/over_60.xes')


In [ ]:
len(under_60)

In [ ]:
len(over_60)

In [ ]:
pd.DataFrame(over_60[0])

In [ ]:
def calc_activity_values(log):
    activity_values = {}
    for trace in log:
        df = pd.DataFrame(trace)
        activities = df["concept:name"].unique()
        for activity in activities:
            if activity not in activity_values:
                activity_values[activity] = {}

            measurements = df.columns
            for measurement in measurements:
                measurement_vals = df[df["concept:name"] == activity][measurement]
                if is_numeric_dtype(measurement_vals):
                    mean_val = np.nanmean(measurement_vals)
                    # todo: exclude nan?
                    if not np.isnan(mean_val):
                        if measurement in activity_values[activity]:
                            activity_values[activity][measurement].append(mean_val)
                        else:
                            activity_values[activity][measurement] = [mean_val]
    return activity_values

In [ ]:
def calculate_mwu_batch(i1, i2):
    df_mwu = pd.DataFrame(columns=["Activity", "Measurement", "U-val", "alternative", "p-val", "RBC", "CLES"])
    for activity in o6.keys():
        if activity in u6:
            for measurement in o6[activity].keys():
                if measurement in u6[activity]:
                    result = pg.mwu(o6[activity][measurement], u6[activity][measurement])
                    result["Activity"] = activity
                    result["Measurement"] = measurement
                    #result.set_index(["Activity", "Measurement"], inplace=True)

                    df_mwu = pd.concat([df_mwu, result])
    df_mwu["RBC_abs"] = df_mwu["RBC"].abs()
    df_mwu = df_mwu.sort_values(["Activity", "RBC_abs"], ascending=False)
    return df_mwu

In [ ]:
o6 = calc_activity_values(over_60)
u6 = calc_activity_values(under_60)

In [ ]:
df_mwu = calculate_mwu_batch(o6, u6)

In [ ]:
df_mwu.groupby("Activity").head(5)
# add sample size
# filter p value < 0.05
# p value correction: divide by # of values pro cohorte (Bonferroni Correction)

# a b a vs a b
# Varianten einzeln betrachten

# alle Attribute, die nur in emergency department sind, sind case attribute 
# -> ist bekannt
# Emergency Department Attribute weglassen (dort wird noch nichts aufgenommen)

# next steps:
# coefficient of variation (only on ratio scaled attributes)
# eher zwischen den Aktivitäten anschauen über stat. tests

# wie funktioniert process comparator in PROM?

# Daten aus Emergency Department rausziehen aus dem Datensatz (macht Jonas für uns)